## Preprocessing --GoDaddy

In [2]:
from constants import *
import pandas as pd

In [3]:
import pandas as pd

In [4]:
df_census= pd.read_csv(os.path.join(DATA_DIR, 'census_starter.csv'))

new_names_map= {}
for year in [2017,2018, 2019, 2020, 2021]:
    for attribute in ['pct_bb','pct_college', 'pct_foreign_born','pct_it_workers','median_hh_inc']:
        new_names_map[f"{attribute}_{str(year)}"]= f"{attribute}_{str(year+2)}"
        print("Renaming ",f"{attribute}_{str(year)}"," to",f"{attribute}_{str(year+2)}")
df_census= df_census.rename(columns=new_names_map)
df_census.head()
df_census.to_csv(os.path.join(DATA_DIR, 'census_starter_shifted.csv'))

Renaming  pct_bb_2017  to pct_bb_2019
Renaming  pct_college_2017  to pct_college_2019
Renaming  pct_foreign_born_2017  to pct_foreign_born_2019
Renaming  pct_it_workers_2017  to pct_it_workers_2019
Renaming  median_hh_inc_2017  to median_hh_inc_2019
Renaming  pct_bb_2018  to pct_bb_2020
Renaming  pct_college_2018  to pct_college_2020
Renaming  pct_foreign_born_2018  to pct_foreign_born_2020
Renaming  pct_it_workers_2018  to pct_it_workers_2020
Renaming  median_hh_inc_2018  to median_hh_inc_2020
Renaming  pct_bb_2019  to pct_bb_2021
Renaming  pct_college_2019  to pct_college_2021
Renaming  pct_foreign_born_2019  to pct_foreign_born_2021
Renaming  pct_it_workers_2019  to pct_it_workers_2021
Renaming  median_hh_inc_2019  to median_hh_inc_2021
Renaming  pct_bb_2020  to pct_bb_2022
Renaming  pct_college_2020  to pct_college_2022
Renaming  pct_foreign_born_2020  to pct_foreign_born_2022
Renaming  pct_it_workers_2020  to pct_it_workers_2022
Renaming  median_hh_inc_2020  to median_hh_inc_2022


## Reformatting census data

In [5]:
years= [2019,2020,2021,2022,2023]
attributes= ['pct_bb','pct_college', 'pct_foreign_born','pct_it_workers','median_hh_inc']
##Adding the attributes to the dataframe df_train, for month take the value of the corresponding year and cfips
col_map = {}
col_map["year"] = []
for year in years:
    col_map["year"] += [year] * df_census.shape[0]
for attribute in attributes:
    col_map[attribute] = []
    for year in years:
        col_map[attribute] += list(df_census[attribute+ '_' + str(year)])
col_map["cfips"] = list(df_census["cfips"]) * len(years)
df_census_reshaped = pd.DataFrame(col_map)


# Fill in missing values with mean of the column for each cfips
for column in df_census_reshaped.columns:
    if column not in ['year', 'cfips']:
        df_census_reshaped[column] = df_census_reshaped.groupby('cfips')[column].apply(lambda x: x.fillna(x.mean()))

In [6]:
df_census_interpolated= df_census_reshaped.copy()
#Create a column date and add rows for each month then interpolate the values for each month
df_census_interpolated["first_day_of_month"]=pd.to_datetime(df_census_interpolated["year"], format='%Y')

df_census_interpolated["first_day_of_month"]=pd.to_datetime(df_census_interpolated["first_day_of_month"].dt.strftime("%Y-%m-%d"))

#Add rows for each month put nan for the attributes of new rows
df_months_list=[]
for month_num in range(2,13):
    df_census_new_date=df_census_interpolated.copy()

    #Set the month num of the date
    df_census_new_date["first_day_of_month"]=df_census_new_date["first_day_of_month"]+pd.DateOffset(months=month_num-1)

    #Set all the attributes to nan
    for attribute in attributes:
        df_census_new_date[attribute]=None

    df_months_list.append(df_census_new_date)

##Concatenate the new rows to the original dataframe
df_census_interpolated=pd.concat(df_months_list+ [df_census_interpolated], ignore_index=True)

#INterpolate the values for each month
df_census_interpolated=df_census_interpolated.sort_values(by=['cfips', 'first_day_of_month'])
interpolated_attributes=df_census_interpolated.groupby('cfips').apply(lambda x: x[attributes].interpolate(method='linear',limit_direction='both'))
df_census_interpolated[attributes]=interpolated_attributes
df_census_interpolated.to_csv(os.path.join(DATA_DIR, 'census_interpolated.csv'), index=False)

In [7]:
df_census_interpolated

,year,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc,cfips,first_day_of_month
172810,2019,76.600000,14.500000,2.100000,1.300000,55317.000000,1001,2019-01-01
0,2019,76.791667,14.616667,2.091667,1.283333,55606.083333,1001,2019-02-01
15710,2019,76.983333,14.733333,2.083333,1.266667,55895.166667,1001,2019-03-01
31420,2019,77.175000,14.850000,2.075000,1.250000,56184.250000,1001,2019-04-01
47130,2019,77.366667,14.966667,2.066667,1.233333,56473.333333,1001,2019-05-01
...,...,...,...,...,...,...,...,...
109969,2023,81.300000,13.900000,1.600000,0.000000,65566.000000,56045,2023-08-01
125679,2023,81.300000,13.900000,1.600000,0.000000,65566.000000,56045,2023-09-01
141389,2023,81.300000,13.900000,1.600000,0.000000,65566.000000,56045,2023-10-01
157099,2023,81.300000,13.900000,1.600000,0.000000,65566.000000,56045,2023-11-01


In [8]:
df_census_reshaped_copy=df_census_reshaped.copy()
df_census_reshaped_copy.sort_values(by=['cfips', 'year'])

,year,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc,cfips
0,2019,76.6,14.5,2.1,1.3,55317.0,1001
3142,2020,78.9,15.9,2.0,1.1,58786.0,1001
6284,2021,80.6,16.1,2.3,0.7,58731.0,1001
9426,2022,82.7,16.7,2.3,0.6,57982.0,1001
12568,2023,85.5,16.4,2.1,1.1,62660.0,1001
...,...,...,...,...,...,...,...
3141,2019,71.1,14.1,3.8,0.6,59605.0,56045
6283,2020,73.3,13.5,4.1,0.6,52867.0,56045
9425,2021,76.8,13.4,1.7,0.0,57031.0,56045
12567,2022,79.7,12.7,2.3,0.0,53333.0,56045


## Train test splitting of the data df_census_reshaped

In [9]:
from sklearn.model_selection import train_test_split
## 80/20 split of df_census_reshaped and save to train_census_ae and train_census_ae_val
df_train_census_ae, df_train_census_ae_val = train_test_split(df_census_reshaped, test_size=0.2, random_state=42)
df_train_census_ae.to_csv(os.path.join(DATA_DIR, 'census_interpolated_train.csv'), index=False)
df_train_census_ae_val.to_csv(os.path.join(DATA_DIR, 'census_interpolated_val.csv'), index=False)




In [10]:

df_train= pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_train['year']=pd.to_datetime(df_train['first_day_of_month'].str.split("-", expand=True)[0])
df_train.head()


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019-01-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019-01-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019-01-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019-01-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019-01-01


In [11]:
df_train["microbusiness_density"].mean()
df_train["microbusiness_density"].std()

4.991086802428146

In [12]:
df_train.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [13]:

df_census_reshaped["year"]=pd.to_datetime(df_census_reshaped["year"], format='%Y')

In [14]:
df_train_update= pd.merge(df_train, df_census_reshaped, how='left', on=['year', 'cfips'])